# SVM localization for 5G NSA

In [24]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
nr_df = pd.read_csv('/content/drive/MyDrive/localization/5G_fingerprint_DB_grid.csv', sep=',')
nr_df

,Timestamp,Lat,Lon,BMAVAL1,BMAXML1,BMAXML2,BMAXML3,BMAXMQ1,BMAXMQ2,BMAXMQ3,...,TBTS5L3,TBTS5Q1,TBTS5Q2,TBTS5Q3,TBTS6K1,TBTS6K2,TBTS6L1,TBTS6L2,TBTS6Q1,TBTS6Q2
0,1701853010,49.227139,16.575808,0.0,0.0,0.0,0.0,0,0,0,...,0,0,0,0,0.0,0.0,0,0,0,0
1,1701853011,49.227137,16.575807,0.0,0.0,0.0,0.0,0,0,0,...,0,0,0,0,0.0,0.0,0,0,0,0
2,1701853012,49.227134,16.575806,0.0,0.0,0.0,0.0,0,0,0,...,0,0,0,0,0.0,0.0,0,0,0,0
3,1701853013,49.227133,16.575806,0.0,0.0,0.0,0.0,0,0,0,...,0,0,0,0,0.0,0.0,0,0,0,0
4,1701853014,49.227133,16.575805,0.0,0.0,0.0,0.0,0,0,0,...,0,0,0,0,0.0,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29267,1701951192,49.230384,16.571568,0.0,0.0,0.0,0.0,0,0,0,...,0,0,0,0,0.0,0.0,0,0,0,0
29268,1701951194,49.230385,16.571568,0.0,0.0,0.0,0.0,0,0,0,...,0,0,0,0,0.0,0.0,0,0,0,0
29269,1701951196,49.230386,16.571568,0.0,0.0,0.0,0.0,0,0,0,...,0,0,0,0,0.0,0.0,0,0,0,0
29270,1701951198,49.230387,16.571568,0.0,0.0,0.0,0.0,0,0,0,...,0,0,0,0,0.0,0.0,0,0,0,0


In [33]:
X = nr_df.iloc[:, 3:]
y = nr_df.iloc[:, 1:3]

In [34]:
# Scaling X and y
from sklearn.preprocessing import StandardScaler

sc_X = StandardScaler()
sc_y = StandardScaler()
X = sc_X.fit_transform(X)
y = sc_y.fit_transform(y)

In [35]:
import sklearn
from sklearn.model_selection import train_test_split

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=100, shuffle=True, test_size=0.2)

## SVM

In [37]:
from sklearn.svm import SVR

In [38]:
regressor_lat = SVR(kernel='rbf')
regressor_lat.fit(X_train, y_train[:,0])

SVR()

In [39]:
regressor_lon = SVR(kernel='rbf')
regressor_lon.fit(X_train, y_train[:,1])

SVR()

In [40]:
y_pred_lat = regressor_lat.predict(X_test)
y_pred_lon = regressor_lon.predict(X_test)

In [41]:
y_pred_lat

array([ 0.82972511, -1.5166185 ,  1.64320422, ...,  0.35616754,
        1.76734407, -0.28730497])

In [42]:
y_pred_lon

array([ 0.14776762, -0.64242558,  0.12699359, ...,  0.68679541,
       -0.56864358, -1.35372991])

In [43]:
# Determine accuracy uisng 𝑅^2
from sklearn.metrics import r2_score

In [44]:
# R^2 score
score_lat = r2_score(y_test[:,0], y_pred_lat)

print("R^2 lat - {}%".format(round(score_lat, 5) *100))

R^2 lat - 97.975%


In [45]:
score_lon = r2_score(y_test[:,1], y_pred_lon)

print("R^2 lon - {}%".format(round(score_lon, 5) *100))

R^2 lon - 97.57400000000001%


In [46]:
score_combine = (score_lat + score_lon)/2

print("R^2 combine - {}%".format(round(score_combine, 5) *100))

R^2 combine - 97.775%


## MSE and RMSE

In [47]:
y_test_fin = sc_y.inverse_transform(y_test)
y_test_fin

array([[49.212642, 16.617046],
       [49.163933, 16.597515],
       [49.226867, 16.61717 ],
       ...,
       [49.201525, 16.63296 ],
       [49.225454, 16.589678],
       [49.192302, 16.563921]])

In [48]:
y_pred_join = np.stack((y_pred_lat, y_pred_lon), axis=1)
y_pred_join

array([[ 0.82972511,  0.14776762],
       [-1.5166185 , -0.64242558],
       [ 1.64320422,  0.12699359],
       ...,
       [ 0.35616754,  0.68679541],
       [ 1.76734407, -0.56864358],
       [-0.28730497, -1.35372991]])

In [49]:
y_pred_fin = sc_y.inverse_transform(y_pred_join)
y_pred_fin

array([[49.21130404, 16.61434504],
       [49.17242634, 16.58604419],
       [49.22478297, 16.61360102],
       ...,
       [49.20345744, 16.63365038],
       [49.22683991, 16.5886867 ],
       [49.19279543, 16.56056875]])

In [50]:
from sklearn.metrics import mean_squared_error

In [51]:
mse_svm = mean_squared_error(y_test_fin, y_pred_fin)
rmse_svm = mean_squared_error(y_test_fin, y_pred_fin, squared = False)

print(mse_svm)
print(rmse_svm)

1.851157066895767e-05
0.003982097285800587


## Distance error

In [52]:
from math import acos, sin, cos, radians

# Defining the function to calculate geographical distance from coordinates
def geo_distance(true_coord, pred_coord):
  geo_dist = np.empty(len(true_coord), dtype=float)
  for i in range(0, len(true_coord)):
    geo_dist[i] = acos(sin(radians(true_coord[i, 0]))*sin(radians(pred_coord[i, 0]))+cos(radians(true_coord[i, 0]))*cos(radians(pred_coord[i, 0]))*cos(radians(pred_coord[i, 1]) - radians(true_coord[i, 1]))) * 6371
  return geo_dist


In [53]:
geo_dist_svm = geo_distance(y_test_fin, y_pred_fin)
geo_dist_svm

array([0.24622468, 1.25993262, 0.34766741, ..., 0.2206539 , 0.17009015,
       0.24970358])

In [55]:
geo_dist_svm_m = [i * 1000 for i in geo_dist_svm]

In [57]:
# Get average of a list
def Average(lst):
    return sum(lst) / len(lst)

average_error_svm = Average(geo_dist_svm_m)

# Printing average of the list
print("Average SVM error [m] =", round(average_error_svm, 5))
print("Max SVM error [m] = ", np.max(geo_dist_svm_m))
print("Min SVM error [m] = ", np.min(geo_dist_svm_m))

Average SVM error [m] = 293.24933
Max SVM error [m] =  5795.870014510308
Min SVM error [m] =  0.13425878593145296


# Cross check

In [69]:
cross_check_df = pd.read_csv('/content/drive/MyDrive/localization/5G_fingerprint_DB_circle.csv', sep=',')
cross_check_df

,Timestamp,Lat,Lon,BMAVAL1,BMAXML1,BMAXML2,BMAXML3,BMAXMQ1,BMAXMQ2,BMAXMQ3,...,TBTS5L3,TBTS5Q1,TBTS5Q2,TBTS5Q3,TBTS6K1,TBTS6K2,TBTS6L1,TBTS6L2,TBTS6Q1,TBTS6Q2
0,1701439521,49.227657,16.575590,0.0,0,0,0,0,0,0,...,0,0,0,0,0.0,0.0,0,0,0,0
1,1701439522,49.227656,16.575592,0.0,0,0,0,0,0,0,...,0,0,0,0,0.0,0.0,0,0,0,0
2,1701439523,49.227656,16.575593,0.0,0,0,0,0,0,0,...,0,0,0,0,0.0,0.0,0,0,0,0
3,1701439524,49.227654,16.575595,0.0,0,0,0,0,0,0,...,0,0,0,0,0.0,0.0,0,0,0,0
4,1701439525,49.227653,16.575596,0.0,0,0,0,0,0,0,...,0,0,0,0,0.0,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7368,1701447326,49.197588,16.608551,0.0,0,0,0,0,0,0,...,0,0,0,0,0.0,0.0,0,0,0,0
7369,1701447327,49.197582,16.608546,0.0,0,0,0,0,0,0,...,0,0,0,0,0.0,0.0,0,0,0,0
7370,1701447328,49.197578,16.608546,0.0,0,0,0,0,0,0,...,0,0,0,0,0.0,0.0,0,0,0,0
7371,1701447329,49.197579,16.608549,0.0,0,0,0,0,0,0,...,0,0,0,0,0.0,0.0,0,0,0,0


In [71]:
X_check = cross_check_df.iloc[:, 3:]
y_check = cross_check_df.iloc[:, 1:3]
y_check_ori = cross_check_df.iloc[:, 1:3]

In [72]:
# Scaling cross check data

sc_X_cross = StandardScaler()
sc_y_cross = StandardScaler()
X_check = sc_X_cross.fit_transform(X_check)
y_check = sc_y_cross.fit_transform(y_check)

In [73]:
y_cross_lat = regressor_lat.predict(X_check)
y_cross_lon = regressor_lon.predict(X_check)

In [76]:
# R^2 score
score_lat_cross = r2_score(y_check[:,0], y_cross_lat)
print("R^2 lat cross check - {}%".format(round(score_lat_cross, 5) *100))

score_lon_cross = r2_score(y_check[:,1], y_cross_lon)
print("R^2 lon cross check - {}%".format(round(score_lon_cross, 5) *100))

score_combine_cross = (score_lat_cross + score_lon_cross)/2
print("R^2 combine cross check - {}%".format(round(score_combine_cross, 5) *100))

R^2 lat cross check - 85.433%
R^2 lon cross check - 63.585%
R^2 combine cross check - 74.509%


## MSE and RMSE

In [77]:
y_cross_join = np.stack((y_cross_lat, y_cross_lon), axis=1)
y_cross_join

array([[ 1.58266719, -0.87066754],
       [ 1.56683725, -0.86143191],
       [ 1.58720403, -0.87271404],
       ...,
       [ 0.11531148,  0.11266822],
       [ 0.13727054, -0.12665684],
       [ 0.08321796, -0.18976883]])

In [78]:
y_cross_fin = sc_y_cross.inverse_transform(y_cross_join)
y_cross_fin

array([[49.22611506, 16.59348806],
       [49.22581562, 16.59368198],
       [49.22620088, 16.59344509],
       ...,
       [49.19835884, 16.6141346 ],
       [49.19877421, 16.60910963],
       [49.19775177, 16.6077845 ]])

In [79]:
y_cross_fin.shape

(7373, 2)

In [80]:
mse_svm_cross = mean_squared_error(y_check_ori, y_cross_fin)
rmse_svm_cross = mean_squared_error(y_check_ori, y_cross_fin, squared = False)

print(mse_svm_cross)
print(rmse_svm_cross)

0.00010632838417209201
0.009944889510097765


## Distance cross check

In [82]:
y_check_arr = y_check_ori.to_numpy()

In [83]:
geo_dist_svm_cross = geo_distance(y_check_arr, y_cross_fin)
geo_dist_svm_cross

array([1.31097359, 1.32949756, 1.30643236, ..., 0.41524641, 0.13900437,
       0.05861284])

In [84]:
geo_dist_svm_cross_m = [i * 1000 for i in geo_dist_svm_cross]
geo_dist_svm_cross_m

[1310.9735869822512,
 1329.4975572239127,
 1306.432360398844,
 1329.2389898425636,
 1327.791518287261,
 1335.3493747240805,
 1330.0626832470668,
 1335.206892184109,
 1333.7200479737712,
 1325.1061160270692,
 1328.7972916604342,
 1319.191252358028,
 1322.9919375969464,
 1328.5102647859796,
 1316.3834595870264,
 1334.8343894387592,
 1324.2858051170065,
 1314.1413834893722,
 1317.6669456502452,
 1315.835030978352,
 1327.696221708836,
 1333.4057466190957,
 1332.0246136097026,
 1331.3722216979334,
 1322.3611320399336,
 1319.2194408770642,
 1327.461246606842,
 1326.1682174981693,
 1328.7429822099944,
 1322.9384525972948,
 1321.0297119812753,
 1286.9033887877986,
 1241.8663391933012,
 1244.4899852651824,
 1244.4260612991725,
 1244.4153931040819,
 1209.468141804716,
 1199.2123255697647,
 1192.4503505419862,
 1308.2867241021856,
 1294.639521225543,
 1298.1394312300017,
 1180.0956316604227,
 1199.2926676041984,
 1192.1316528454026,
 1273.914126311041,
 1285.551321719309,
 1288.5374023116165,
 12

In [85]:
average_error_svm_cross = Average(geo_dist_svm_cross_m)

# Printing average of the list
print("Average SVM error cross check [m] =", round(average_error_svm_cross, 5))
print("Max SVM error cross check [m] = ", np.max(geo_dist_svm_cross_m))
print("Min SVM error cross check [m] = ", np.min(geo_dist_svm_cross_m))

Average SVM error cross check [m] = 991.41841
Max SVM error cross check [m] =  5544.686105714775
Min SVM error cross check [m] =  11.235302731272798
